In [7]:
import numpy as np
import numpy.testing as npt
import pymc as pm
from pymc.math import cartesian, kronecker

In [9]:
X1 = np.linspace(0, 1, 3)[:, None]
X21 = np.linspace(0, 1, 5)[:, None]
X22 = np.linspace(0, 1, 4)[:, None]
X2 = cartesian(X21, X22)
X = cartesian(X1, X21, X22)
X1.shape, X21.shape, X22.shape, X2.shape, X.shape

((3, 1), (5, 1), (4, 1), (20, 2), (60, 3))

In [23]:
X1.shape, X2.shape, X.shape

((3, 1), (20, 2), (60, 3))

In [15]:
with pm.Model() as model:
    cov1 = (
        3
        + pm.gp.cov.ExpQuad(1, 0.1)
        + pm.gp.cov.ExpQuad(1, 0.1) * pm.gp.cov.ExpQuad(1, 0.1)
    )
    cov2 = pm.gp.cov.ExpQuad(1, 0.1) * pm.gp.cov.ExpQuad(2, 0.1)
    cov = pm.gp.cov.Kron([cov1, cov2])

K = cov(X).eval()

K_true = kronecker(cov1(X1).eval(), cov2(X2).eval()).eval()
npt.assert_allclose(K_true, K)

In [16]:
cov1(X1).eval().shape, cov2(X2).eval().shape

((3, 3), (20, 20))

In [17]:
X.shape, cov(X).eval().shape

((60, 3), (60, 60))

### Test Coregion and Kron kernels

In [29]:
X1 = np.linspace(0, 1, 3)[:, None]
X2 = np.linspace(0, 1, 5)[:, None]
X1.shape, X2.shape

((3, 1), (5, 1))

In [30]:
X = pm.math.cartesian(X1, X2)
X.shape

(15, 2)

In [37]:
with pm.Model() as model:
    # Kernel: K_1(x,x')
    ell = pm.Gamma("ell", alpha=2, beta=0.5)
    eta = pm.Gamma("eta", alpha=3, beta=1)
    cov = eta**2 * pm.gp.cov.ExpQuad(input_dim=1, ls=ell)

    # Coregion B matrix: K_2(o,o')
    W = pm.Normal("W", mu=0, sigma=3, shape=(3,2), initval=np.random.randn(3,2))
    kappa = pm.Gamma("kappa", alpha=1.5, beta=1, shape=3)
    coreg = pm.gp.cov.Coregion(input_dim=1, kappa=kappa, W=W)
    
    kron = pm.gp.cov.Kron([cov, coreg])

In [45]:
# Check shape
cov(X1).eval().shape, coreg(X2).eval().shape, kron(X).eval().shape

((3, 3), (5, 5), (15, 15))

In [43]:
npt.assert_allclose(kron(X).eval(), pm.math.kronecker(cov(X1).eval(), coreg(X2).eval()).eval())